In [1]:
import os 
import sys
import pickle 
import hyperspy.api as hs
sys.path.append("..")
import tomondt
from tomondt import Operator, Viewer, utils

import numpy as np
import torch 
import sys
import wandb

sys.path.append("..")

from tqdm import tqdm
from time import time

from dipster.solver import Solver
from dipster import alignments, tomo
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
import pandas as pd
from collections.abc import Iterable

import plotly
import plotly.express as px
import plotly.graph_objs as go
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from PIL import Image
import shutil
import scipy.io as io

path = r'\\ematbyname\emat\TimC\DIPSTER-PublicationData\Publication-Data'
dip_path = os.path.join(path, 'Dipster-recs')
vol_path = os.path.join(path, 'Volumes')
ts_path = os.path.join(path, 'TiltSeries')
rec_path = os.path.join(path, 'Reconstructions')
data_path =  os.path.join(path, 'Results')

In [ ]:
# Perform SIRT reconstruction for Data Set
sample_name = 'Rod-0'
descriptor = None
folder = 'Static'
frame = 100
if descriptor is None:
    ts_name = sample_name
else:
    ts_name = sample_name +'-'+ descriptor 

ts_file = os.path.join(ts_path, ts_name+'-ts.pkl')


ts = pickle.load(open(ts_file, 'rb'))
ts.data  = alignments.rebin(ts.data, 4)
alg = lambda x: tomondt.algorithms.sirt(x,50)
rec_file = os.path.join(rec_path, folder, ts_name+'.vmf')
rec = tomondt.Operator('cupy',0).bp(rec_file,ts,alg,frame)

In [ ]:
#Reconstruct A series of Reconstructions with different window sizes
sample_name = 'Cage-2'
descriptor = 'w90'
folder = 'Optimizations'


start = 55 # default 5
end = 65 # default 100
increment  = 1 # default 5

if descriptor is None:
    ts_name = sample_name
else:
    ts_name = sample_name +'-'+ descriptor 

ts_file = os.path.join(ts_path, ts_name+'-ts.pkl')
ts = pickle.load(open(ts_file, 'rb'))
ts.data  = alignments.rebin(ts.data, 4)
alg = lambda x: tomondt.algorithms.sirt(x,50)
if not os.path.exists(os.path.join(rec_path, folder, ts_name)):
    os.mkdir(os.path.join(rec_path, folder, ts_name))

i=start
while i<=100:
    rec_file = os.path.join(rec_path, folder, ts_name, str(i)+'.vmf' )
    rec = tomondt.Operator('cupy',0).bp(rec_file,ts,alg,i)
    if i == end:
        break
    i += increment

In [79]:
# Evaluates Optimization Frame Optimum 
sample_name = 'Cage-0'
descriptor = 'w90'
if descriptor is None:
    ts_name = sample_name
else:
    ts_name = sample_name +'-'+ descriptor 
opt_folder = os.path.join(rec_path, 'Optimizations', ts_name)

vol = tomondt.load_vmf(os.path.join(vol_path, sample_name+'.vmf'))
x = np.zeros(len(os.listdir(opt_folder)))
y = np.zeros(len(os.listdir(opt_folder)))
read_files = 0
for file in os.listdir(opt_folder):
    frame = int(file.split('.')[0])
    rec_file = os.path.join(opt_folder, file)
    rec = tomondt.load_vmf(rec_file) 
    psnr_val = 0
    for i in rec.times:
        temp_array = i - vol.times
        comp_index = np.argmin(abs(temp_array))
        if isinstance(comp_index, Iterable):
            vol_data = np.zeros_like(vol.read_record(comp_index[0],False))
            for j in comp_index:
                vol_data += vol.read_record(j,False)  
            vol_data = vol_data/len(comp_index)
        else:
            vol_data = vol.read_record(comp_index,False)
        vol_data = alignments.rebin(vol_data, 4, True)
        rec_data = rec.read_record(i)
        psnr_val += psnr(vol_data, rec_data, data_range=1.0)
    psnr_val = psnr_val/len(rec.times)
    x[read_files] = frame
    y[read_files] = psnr_val
    read_files += 1
indices = np.argsort(x)
x = x[indices]
y = y[indices]

data_file  = os.path.join(data_path, ts_name+'-optimization.mat')
if os.path.exists(data_file):
    os.remove(data_file)
io.savemat(data_file, {"frames": x.T, "psnr": y.T})
fig = px.line(x=x, y=y, title="Simple Line Plot")
fig.show()
    

FileNotFoundError: [WinError 3] The system cannot find the path specified: '\\\\ematbyname\\emat\\TimC\\DIPSTER-PublicationData\\Publication-Data\\Reconstructions\\Optimizations\\Cage-0-w90'

In [19]:
# Determine Missing Wedge Contribution PSNR and SSIM for DataSet
sample_name = 'Cage-0'
descriptor = 'w90'
voltype = 0 # 0 for SIRT 1 for DIPSTER
folder = 'MissingWedge'
slice_index = 64

if descriptor is None:
    ts_name = sample_name
else:
    ts_name = sample_name +'-'+ descriptor 

vol_file = os.path.join(vol_path, sample_name+'.vmf')
vol = tomondt.load_vmf(vol_file)
match voltype:
    case 0:
        suffix = '-sirt'
        suffix_underscored = '_sirt'
        rec_file = os.path.join(rec_path, folder, ts_name+'.vmf')
    case 1:
        suffix = '-dipster'
        suffix_underscored = '_dipster'
        for file in os.listdir(os.path.join(dip_path, folder)):
            if ts_name in file:
                rec_file = os.path.join(dip_path, folder, file)
                break
    case _:
        raise ValueError('Invalid Volume Type')
    
    
rec = tomondt.load_vmf(rec_file)

ssims = np.zeros(len(rec.times))
psnrs = np.zeros(len(rec.times))
mwedge = np.zeros(len(rec.times))
times = rec.times

read_index  = 0
for i in rec.times:
    rec_data = rec.read_record(i)
    temp_array = i - vol.times
    comp_index = np.argmin(abs(temp_array))
    if isinstance(comp_index, Iterable):
        vol_data = np.zeros_like(vol.read_record(comp_index[0],False))
        for j in comp_index:
            vol_data += vol.read_record(j,False)  
        vol_data = vol_data/len(comp_index)
    else:
        vol_data = vol.read_record(comp_index,False)
    vol_data = alignments.rebin(vol_data, 4, True)   
    ssims[read_index] = ssim(vol_data, rec_data, data_range=1.0)
    psnrs[read_index] = psnr(vol_data, rec_data, data_range=1.0)
    mw2 = psnr(vol_data[:,slice_index,:] , rec_data[:,slice_index,:], data_range=1.0)
    mw1 = psnr(vol_data[:,:,slice_index], rec_data[:,:,slice_index], data_range=1.0)
    mwedge[read_index] = (mw2/mw1)*100
    read_index += 1


data_file  = os.path.join(data_path, ts_name+suffix+'-true.mat')
if os.path.exists(data_file):
    os.remove(data_file)
    
data_dict = {}
data_dict['times'] = times
data_dict['psnr'] = psnrs
data_dict['ssim'] = ssims
data_dict['mwedge'] = mwedge
data_dict['mwedge_average'] = np.mean(mwedge)
data_dict['ssims_average'] = np.mean(ssims)
data_dict['psnrs_average'] = np.mean(psnrs)
data_struct = {}
data_struct[ts_name.replace('-','_')+suffix_underscored] = data_dict
io.savemat(data_file, data_struct)


In [ ]:
# For getting data of DIPSTER reconstruction quality
path = r'\\ematbyname\emat\TimC\DIPSTER-PublicationData\Publication-Data'
rec_path = r'\\ematbyname\emat\TimC\DIPSTER-PublicationData\Publication-Data\Dipster-recs'
vol_path = r'\\ematbyname\emat\TimC\DIPSTER-PublicationData\Publication-Data\volumes'

sample_name = 'Cube-0'

df = pd.DataFrame(columns=['ssim', 'psnr'])
ref_name = os.path.join(vol_path, sample_name+'.vmf')
for structure in os.listdir(rec_path):
    if (sample_name  in structure):
        df = pd.DataFrame(columns=['ssim', 'psnr'])
        rec_name = os.path.join(rec_path, structure)
        rec = tomondt.load_vmf(rec_name)
        ref = tomondt.load_vmf(ref_name)
        for i in rec.times:
            ref_data = ref.read_record(i)
            ref_data = alignments.rebin(ref_data, 4, True)
            rec_data = rec.read_record(i)
            data_row = {'ssim': ssim(ref_data, rec_data, data_range=1.0), 'psnr': psnr(ref_data, rec_data, data_range=1.0)} 
            df = pd.concat([df, pd.DataFrame([data_row])], ignore_index=True)
        df.to_csv(os.path.join(path, sample_name+'dipster.csv'))
        print(df)

In [ ]:

# Where the data is stored
path = r'\\ematbyname\emat\TimC\DIPSTER-PublicationData\Publication-Data'
ts_path = r'\\ematbyname\emat\TimC\DIPSTER-PublicationData\Publication-Data\TiltSeries'
rec_path = r'\\ematbyname\emat\TimC\DIPSTER-PublicationData\Publication-Data\Reconstructions'
vol_path = r'\\ematbyname\emat\TimC\DIPSTER-PublicationData\Publication-Data\volumes'



for item in os.listdir(ts_path):
    ts_name = os.path.join(ts_path, item)
    ts = pickle.load(open(ts_name, 'rb'))
    ts.data  = alignments.rebin(ts.data, 4)
    for ref_item in os.listdir(vol_path):
       sample_name = ref_item.split('.')[0]
       if sample_name in ts_name and 'Rod' in sample_name:
           df = pd.DataFrame(columns=['psnr', 'framesize'])
           for i in range(100):
               if (i % 2 != 0 and i > 10) or i == 100:
                    alg = lambda x: tomondt.algorithms.sirt(x,50)
                    fname = os.path.join(rec_path, sample_name+'_'+str(i)+'.vmf')
                    rec = tomondt.Operator('cupy',0).bp(fname,ts,alg,i)
                    rec = tomondt.load_vmf(fname)
                    ref = tomondt.load_vmf(os.path.join(vol_path,ref_item))
                    psnr_val = 0 
                    n = 0
                    for j in range(len(rec.times)):
                        for k in range(len(ref.times)):
                            if rec.times[j] == ref.times[k]:
                                rec_data = rec.read_record(rec.times[j])
                                ref_data = ref.read_record(ref.times[k])
                                ref_data = alignments.rebin(ref_data, 4, True)
                                psnr_val += psnr(ref_data, rec_data, data_range=1.0)
                                n += 1
                    data_row = {'psnr': psnr_val/n, 'framesize': i} 
                    df = pd.concat([df, pd.DataFrame([data_row])], ignore_index=True)
                    df.to_csv(os.path.join(path, sample_name+'_optimizing_sirt_psnr.csv'))
                    print(sample_name, i, psnr_val/n)


In [ ]:
path = r'\\ematbyname\emat\TimC\DIPSTER-PublicationData\Publication-Data'
ts_path = r'\\ematbyname\emat\TimC\DIPSTER-PublicationData\Publication-Data\TiltSeries'
rec_path = r'\\ematbyname\emat\TimC\DIPSTER-PublicationData\Publication-Data\Reconstructions'
vol_path = r'\\ematbyname\emat\TimC\DIPSTER-PublicationData\Publication-Data\Volumes'

ref_name = os.path.join(vol_path, 'Rod-0.vmf') 
rec_name = os.path.join(rec_path, 'Rod-0_99.vmf')
ref = tomondt.load_vmf(ref_name)
rec = tomondt.load_vmf(rec_name)
df = pd.DataFrame(columns=['index','ssim', 'psnr'])
for i in range(len(ref.times)):
    for j in range(len(rec.times)):
        if ref.times[i]==rec.times[j]:
            ref_data = ref.read_record(ref.times[i])
            ref_data = alignments.rebin(ref_data, 4, True)
            rec_data = rec.read_record(rec.times[j])
            data_row = {'index': i, 'ssim': ssim(ref_data, rec_data, data_range=1.0), 'psnr': psnr(ref_data, rec_data, data_range=1.0)} 
            print(data_row)
            df = pd.concat([df, pd.DataFrame([data_row])], ignore_index=True)
df.to_csv(os.path.join(path, 'Op-Rod_0.csv'))



In [ ]:
path = r'\\ematbyname\emat\TimC\DIPSTER-PublicationData\Publication-Data'
ts_path = r'\\ematbyname\emat\TimC\DIPSTER-PublicationData\Publication-Data\TiltSeries'
rec_path = r'\\ematbyname\emat\TimC\DIPSTER-PublicationData\Publication-Data\Reconstructions'
dip_path = r'\\ematbyname\emat\TimC\DIPSTER-PublicationData\Publication-Data\Dipster-recs'
vol_path = r'\\ematbyname\emat\TimC\DIPSTER-PublicationData\Publication-Data\Volumes'

sample_name = 'Rod-0'
id = '99'
axis = 2
sliceid = 64
ref = tomondt.load_vmf(os.path.join(vol_path, sample_name+'.vmf'))
for item in os.listdir(dip_path):
    if sample_name in item:
        dip = tomondt.load_vmf(os.path.join(dip_path, item))
for item in os.listdir(rec_path):
    if sample_name in item and id in item:
        rec = tomondt.load_vmf(os.path.join(rec_path, item))

ref_image = ref.read_record(ref.times[0])
ref_image = alignments.rebin(ref_image, 4, True)       
dip_image = dip.read_record(dip.times[0])
rec_image = rec.read_record(rec.times[0])
if axis == 0:
    ref_image = ref_image[sliceid,:,:]
    rec_image = rec_image[sliceid,:,:]
    dip_image = dip_image[sliceid,:,:]
elif axis == 1:
    ref_image = ref_image[:,sliceid,:]
    rec_image = rec_image[:,sliceid,:]
    dip_image = dip_image[:,sliceid,:]
else: 
    ref_image = ref_image[:,:,sliceid]
    rec_image = rec_image[:,:,sliceid]
    dip_image = dip_image[:,:,sliceid]
    
psnr_rec = psnr(ref_image, rec_image, data_range=1.0)
psnr_dip = psnr(ref_image, dip_image, data_range=1.0)
ssim_rec = ssim(ref_image, rec_image, data_range=1.0)
ssim_dip = ssim(ref_image, dip_image, data_range=1.0)
print(psnr_rec, psnr_dip, ssim_rec, ssim_dip)
    

In [ ]:
#For measuring Missing Wedge Artefacts
path = r'\\ematbyname\emat\TimC\DIPSTER-PublicationData\Publication-Data'
ts_path = r'\\ematbyname\emat\TimC\DIPSTER-PublicationData\Publication-Data\TiltSeries'
rec_path = r'\\ematbyname\emat\TimC\DIPSTER-PublicationData\Publication-Data\Reconstructions'
dip_path = r'\\ematbyname\emat\TimC\DIPSTER-PublicationData\Publication-Data\Dipster-recs'
vol_path = r'\\ematbyname\emat\TimC\DIPSTER-PublicationData\Publication-Data\Volumes'

sample_name = 'Rod-0'
angles = [10,20,30,40,50,60,70,80,90]

for sample in os.listdir(vol_path):
    if sample_name in sample:
        ref = tomondt.load_vmf(os.path.join(vol_path, sample))
for ts in os.listdir(ts_path):
    for i in angles:
        if sample_name in ts and str(i) in ts:
            tilt_series = pickle.load(open(os.path.join(ts_path, ts), 'rb'))
            rec_name = sample_name+'_'+str(i)+'.vmf'
            rec = tomondt.load_vmf(os.path.join(rec_path, rec_name))
            for t in ref.times:
                

In [ ]:
#For measuring Noise Artefacts
path = r'\\ematbyname\emat\TimC\DIPSTER-PublicationData\Publication-Data'
ts_path = r'\\ematbyname\emat\TimC\DIPSTER-PublicationData\Publication-Data\TiltSeries'
rec_path = r'\\ematbyname\emat\TimC\DIPSTER-PublicationData\Publication-Data\Reconstructions'
dip_path = r'\\ematbyname\emat\TimC\DIPSTER-PublicationData\Publication-Data\Dipster-recs'
vol_path = r'\\ematbyname\emat\TimC\DIPSTER-PublicationData\Publication-Data\Volumes'

sample_name = 'Cage-2'
values = [4]
df = pd.DataFrame(columns=['ssim', 'psnr'])
viewer = tomondt.Viewer()
for sample in os.listdir(vol_path):
    if sample_name in sample:
        ref = tomondt.load_vmf(os.path.join(vol_path, sample))
for ts in os.listdir(ts_path):
    for i in values:
        if sample_name in ts and ('n'+str(i)) in ts:
            tilt_series = pickle.load(open(os.path.join(ts_path, ts), 'rb'))
            tilt_series.data = alignments.rebin(tilt_series.data, 4).astype('float32')
            rec_name = sample_name+'_n'+str(i)+'.vmf'
            rec = tomondt.Operator('cupy',0).bp(os.path.join(rec_path, rec_name), tilt_series, lambda x: tomondt.algorithms.sirt(x,50), 100)
            ref_data,rec_data = alignments.rebin(ref.read_record(ref.times[0]),4,True), rec.read_record(rec.times[0])
            #rec_data = (rec_data - np.min(rec_data))/(np.max(rec_data)-np.min(rec_data))
            viewer.add_image(rec_data, name=rec_name)
            data_row = {'index': i, 'ssim': ssim(ref_data, rec_data, data_range=1.0), 'psnr': psnr(ref_data, rec_data, data_range=1.0)} 
            print(i, data_row)
            df = pd.concat([df, pd.DataFrame([data_row])], ignore_index=True)
df.to_csv(os.path.join(path, 'Noise-Cage_0.csv'))

